In [3]:
import sys
import os
import tweepy
import csv
import time


with open('%s/.tweetcred.txt' % os.environ['HOME'], 'r') as creds:
    ckey, csec, atok, atoksec = creds.readlines()

    
consumer_key = ckey.split()[2]
consumer_secret = csec.split()[2]
access_token = atok.split()[2]
access_token_secret = atoksec.split()[2]


# Replace the API_KEY and API_SECRET with your application's key and secret.
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)


api = tweepy.API(auth, wait_on_rate_limit=True,
                 wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

# Continue with rest of code

In [5]:
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)

In [11]:
la_last_id = -1
chi_last_id = -1
hou_last_id = -1

for i in range(1, 10):
    # If results only below a specific ID are, set max_id to that ID.
    # else default to no upper limit, start from the most recent tweet matching the search query.
    max_id = -1L
    if (max_id <= 0):
        with open('tweets_csvs/tweets_la%d.csv' % i, 'w') as csvfile:
            tweetwriter = csv.writer(csvfile, delimiter = ',')
            for tweet in limit_handled(tweepy.Cursor(api.search,q="*",count=15000,max_id=str(la_last_id - 1),
                                                     geocode="34.0904489,-118.2305523,25km").items(15000)):
                if hasattr(tweet, 'retweeted_status'):
                    rtweet = 1
                    rtweet_txt = tweet.retweeted_status.text.encode('utf-8')
                else:
                    rtweet = 0
                    rtweet_txt = 'NO_RT'
                tweetwriter.writerow([tweet.id_str, tweet.user.screen_name, tweet.user.followers_count,
                                      tweet.user.statuses_count, tweet.text.encode('utf-8'),
                                      rtweet, tweet.retweet_count, rtweet_txt, tweet.favorite_count])
        la_last_id = tweet.id
        time.sleep(3 * 60)
        try:
            with open('tweets_csvs/tweets_chi%d.csv' % i, 'w') as csvfile:
                tweetwriter = csv.writer(csvfile, delimiter = ',')
                for tweet in limit_handled(tweepy.Cursor(api.search,q="*",count=15000,max_id=str(chi_last_id - 1),
                                                         geocode="41.8788493,-87.6275801,25km").items(15000)):
                    if hasattr(tweet, 'retweeted_status'):
                        rtweet = 1
                        rtweet_txt = tweet.retweeted_status.text.encode('utf-8')
                    else:
                        rtweet = 0
                        rtweet_txt = 'NO_RT'
                    tweetwriter.writerow([tweet.id_str, tweet.user.screen_name, tweet.user.followers_count,
                                          tweet.user.statuses_count, tweet.text.encode('utf-8'),
                                          rtweet, tweet.retweet_count, rtweet_txt, tweet.favorite_count])
            chi_last_id = tweet.id
            time.sleep(3 * 60)
        except tweepy.TweepError as e:
            continue
        try:
            with open('tweets_csvs/tweets_hou%d.csv' % i, 'w') as csvfile:
                tweetwriter = csv.writer(csvfile, delimiter = ',')
                for tweet in limit_handled(tweepy.Cursor(api.search,q="*",count=15000,max_id=str(hou_last_id - 1),
                                                         geocode="29.8384701,-95.4465094,25km").items(15000)):
                    if hasattr(tweet, 'retweeted_status'):
                        rtweet = 1
                        rtweet_txt = tweet.retweeted_status.text.encode('utf-8')
                    else:
                        rtweet = 0
                        rtweet_txt = 'NO_RT'
                    tweetwriter.writerow([tweet.id_str, tweet.user.screen_name, tweet.user.followers_count,
                                          tweet.user.statuses_count, tweet.text.encode('utf-8'),
                                          rtweet, tweet.retweet_count, rtweet_txt, tweet.favorite_count])
            hou_last_id = tweet.id
            time.sleep(3 * 60)
        except tweepy.TweepError as e:
            continue
    max_id = tweet.id_str

TweepError: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out.

In [21]:
num_tweets = 15000
cities = {'la': "34.0904489,-118.2305523,25km",
          'chi': "41.8788493,-87.6275801,25km",
          'hou': "29.8384701,-95.4465094,25km"}

#tweetids = {'la': -1, 'chi': -1, 'hou': -1}

max_id = -1L
for i in range(24, 27):
    for city in cities:
        try:
            with open('tweets_csvs/tweets_%s%d.csv' % (city, i), 'w') as csvfile:
                tweetwriter = csv.writer(csvfile, delimiter = ',')
                for tweet in limit_handled(tweepy.Cursor(api.search,q="*",
                                                         count=num_tweets,
                                                         max_id=str(tweetids[city] - 1),
                                                         geocode=cities[city]).items(num_tweets)):
                    if hasattr(tweet, 'retweeted_status'):
                        rtweet = 1
                        rtweet_txt = tweet.retweeted_status.text.encode('utf-8')
                    else:
                        rtweet = 0
                        rtweet_txt = 'NO_RT'
                    tweetwriter.writerow([tweet.id_str, tweet.user.screen_name, tweet.user.followers_count,
                                          tweet.user.statuses_count, tweet.text.encode('utf-8'),
                                          rtweet, tweet.retweet_count, rtweet_txt, tweet.favorite_count])
            tweetids[city] = tweet.id
            time.sleep(3 * 60)
        except tweepy.TweepError as e:
            continue


Rate limit reached. Sleeping for: 188
Rate limit reached. Sleeping for: 22


In [23]:
tweetids

{'chi': 876116451517661186,
 'hou': 876014460477075456,
 'la': 876262576316874753}